In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
from pyproj import CRS
from shapely.geometry import Polygon
from shapely.ops import unary_union

pd.set_option('display.max_rows', None)

In [ ]:
# This notebook:
# 1. Cleans the watersheds shapefile by getting rid of very small polygons on edges
# 2. Creates intermediate catchments (delineation B) using the watersheds shapefile from delineation A
# and the gauge hierarchy

In [3]:
# Define functions
def subtract_catchments(watershed_gdf, current_id, upstream_id):
    # Get the geometry of the two polygons
    polygon1 = watershed_gdf.loc[int(current_id)].geometry
    polygon2 = watershed_gdf.loc[int(upstream_id)].geometry

    # Create a temporary GeoDataFrame with the polygon geometry
    temp_gdf = gpd.GeoDataFrame(geometry=[polygon1], crs=watershed_gdf.crs)

    # Perform the overlay operation
    subtracted_geometry = gpd.overlay(temp_gdf, gpd.GeoDataFrame(geometry=[polygon2], crs=watershed_gdf.crs), how='difference').geometry.values[0]

    # Create a new GeoDataFrame with the subtracted geometry
    subtracted_gdf = gpd.GeoDataFrame(geometry=[subtracted_geometry], crs=watershed_gdf.crs)

    # Check the resulting GeoDataFrame
    return(subtracted_gdf)

def subtract_joined_catchments(watershed_gdf, current_id, joined_catchments_geom):
    polygon1 = watershed_gdf.loc[int(current_id)].geometry
    polygon2 = joined_catchments_geom
    
    # Create a temporary GeoDataFrame with the polygon geometry
    temp_gdf = gpd.GeoDataFrame(geometry=[polygon1], crs=watershed_gdf.crs)

    # Perform the overlay operation
    subtracted_geometry = gpd.overlay(temp_gdf, gpd.GeoDataFrame(geometry=[polygon2], crs=watershed_gdf.crs), how='difference').geometry.values[0]

    # Create a new GeoDataFrame with the subtracted geometry
    subtracted_gdf = gpd.GeoDataFrame(geometry=[subtracted_geometry], crs=watershed_gdf.crs)

    # Check the resulting GeoDataFrame
    return(subtracted_gdf)

In [21]:
watershed_gdf = gpd.read_file(Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\watersheds.shp"))
watershed_gdf.index = watershed_gdf['index']

In [43]:
# Read the original watersheds shapefile
watershed_gdf = gpd.read_file(Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\watersheds.shp"))
watershed_gdf.index = watershed_gdf['index']

# Now we need to clean the watersheds shapefile - some watersheds are of the type "multipolygon"
# We loop through all watersheds and for multipolygons, we select only the largest polygon

# Create an empty list to store the basin geometries
basin_geometries = []
basin_ids = []
# Loop through all polygons in the GeoDataFrame
for index, row in watershed_gdf.iterrows():
    # print(index)
    geometry = row['geometry']
    
    # Check if the geometry is a MultiPolygon
    if geometry.geom_type == 'MultiPolygon':
        # Find the largest polygon within the MultiPolygon
        
        # Initialize variables to store the largest polygon information
        largest_polygon = None
        largest_area = 0

        # Iterate over the individual polygons within the MultiPolygon
        for polygon in geometry.geoms:
            # Calculate the area of the current polygon
            polygon_area = polygon.area
            # Check if the current polygon has a larger area than the previous largest polygon
            if polygon_area > largest_area:
                largest_polygon = polygon
                largest_area = polygon_area

        # Create a GeoSeries with the largest polygon
        largest_polygon_series = gpd.GeoSeries([largest_polygon])
        # Create a GeoDataFrame with the largest polygon
        largest_polygon_gdf = gpd.GeoDataFrame(geometry=largest_polygon_series)
        
        basin_geometries.append(largest_polygon)
    else:
        basin_geometries.append(geometry)
    basin_ids.append(index)
    
# Create a list of tuples containing the polygon geometries and indices
features = [(Polygon(geometry), index) for geometry, index in zip(basin_geometries, basin_ids)]

# Create a GeoDataFrame from the features list
basin_gdf = gpd.GeoDataFrame(features, columns=['geometry', 'index'])

# Specify the CRS
crs = CRS.from_epsg(3057)

# Set the CRS of the GeoDataFrame
basin_gdf.crs = crs

# save as shapefile
savepath = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\watersheds_cleaned_no_multipolygons.shp")
basin_gdf.to_file(savepath)

In [4]:
# Read the updated catchment shapefile back in, as well as a .csv file with the gauge hierarchy
watershed_gdf = gpd.read_file(Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\final_watersheds\Basins_A.shp"))
watershed_gdf.index = watershed_gdf['index']
gauge_df = pd.read_csv(Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\lamah_ice\B_basins_intermediate_all\1_attributes\Gauge_hierarchy.csv"),sep=';')

In [5]:
# Now we loop through and create intermediate catchments
basin_geometries = []  # Initialize a list to store basin geometries
basin_ids = []
for _, row in gauge_df.iterrows():
    upstream_gauge = row["NEXTUPID"]
    downstream_gauge = row["NEXTDOWNID"]
    upstream_gauge = upstream_gauge.split(',')
    if upstream_gauge == ['0']:
        # If there is no upstream gauge, start with the current gauge's catchment
        basin_geom = watershed_gdf[watershed_gdf["index"] == row["ID"]].geometry.iloc[0]
    
    elif len(upstream_gauge)>1:
        print('length is >1')
        number_indices = [int(index) for index in upstream_gauge]
        merged_geometry = watershed_gdf.loc[watershed_gdf['index'].isin(number_indices), 'geometry']
        merged_polygon = unary_union(merged_geometry)
        basin_geom = subtract_joined_catchments(watershed_gdf, row.ID, merged_polygon)

    else:
        upstream_gauge = upstream_gauge[0]
        # Subtract the catchment of the upstream gauge from the basin geometry
        basin_geom = subtract_catchments(watershed_gdf, row.ID, upstream_gauge)
    # If the resulting geometry is a multipolygon, we repeat the process of eliminating small polygons
    if basin_geom.geom_type[0] == 'MultiPolygon':
        # Initialize variables to store the largest and second largest polygon information
        largest_polygon = None
        largest_area = 0
        second_largest_polygon = None
        second_largest_area = 0

        multi_polygon = basin_geom.loc[0, 'geometry']

        # Iterate over the individual polygons within the MultiPolygon
        for polygon in multi_polygon.geoms:
            # Calculate the area of the current polygon
            polygon_area = polygon.area

            # Check if the current polygon has a larger area than the previous largest polygon
            if polygon_area > largest_area:
                second_largest_polygon = largest_polygon
                second_largest_area = largest_area
                largest_polygon = polygon
                largest_area = polygon_area
            # Check if the current polygon has a larger area than the second largest polygon
            elif polygon_area > second_largest_area:
                second_largest_polygon = polygon
                second_largest_area = polygon_area

        if row.ID in [100,88]:
            # Note: This is hardcoded in because watersheds 100 and 88 had other, larger polygons that were not correct.
            basin_geom = second_largest_polygon
        else:
            basin_geom = largest_polygon

    if isinstance(basin_geom, gpd.GeoDataFrame):
        basin_geom = basin_geom.geometry[0]

    basin_geometries.append(basin_geom)  # Add the updated basin geometry to the list
    basin_ids.append(row.ID)
    
# Create a list of tuples containing the polygon geometries and indices

# Assuming you have a list of geometries called 'basin_geometries' and a corresponding list of indices called 'basin_ids'
features = []
for geometry, index in zip(basin_geometries, basin_ids):
    # Check if the geometry is valid before adding it to the features list
    if isinstance(geometry, Polygon):
        features.append((geometry, index))
    else:
        print('The geometry for ID %s is not actually a geometry' % index)
#         print(hi)

# Create the GeoDataFrame
basin_gdf = gpd.GeoDataFrame(features, columns=['geometry', 'index'])

# Specify the CRS
crs = CRS.from_epsg(3057)

# Set the CRS of the GeoDataFrame
basin_gdf.crs = crs

# save as shapefile
savepath = Path(r"C:\Users\hordurbhe\Dropbox\UW\lamah_ice\GIS\watersheds\final_watersheds\intermediate_watersheds_08242023_v8.shp")
basin_gdf.to_file(savepath)


length is >1
length is >1
length is >1
length is >1
length is >1
length is >1
length is >1
length is >1
length is >1
length is >1
length is >1
length is >1
length is >1
length is >1
length is >1
